In [24]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import traceback

In [5]:
 from langchain_google_genai import ChatGoogleGenerativeAI

In [25]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [26]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GEMINI_API_KEY)
llm.invoke("Sing a ballad of LangChain.")

AIMessage(content="**Verse 1:**\nIn realms of code, where data flows,\nA legend arose, LangChain's prose.\nA language vast, with boundless might,\nGuiding AI's path, shining bright.\n\n**Chorus:**\nOh, LangChain, LangChain,\nYour syntax weaves a boundless chain.\nFrom simple words to complex lore,\nYou bridge the gap, forevermore.\n\n**Verse 2:**\nWith every line, a story unfolds,\nUnveiling patterns, secrets yet untold.\nFrom chatbots wise to apps that soar,\nLangChain empowers, forevermore.\n\n**Chorus:**\nOh, LangChain, LangChain,\nYour syntax weaves a boundless chain.\nFrom simple words to complex lore,\nYou bridge the gap, forevermore.\n\n**Bridge:**\nIn classrooms and in labs it reigns,\nInspiring minds and sparking new gains.\nA tool for thought, a pathway wide,\nUnlocking knowledge, side by side.\n\n**Verse 3:**\nThrough natural language, it takes flight,\nConnecting humans with AI's might.\nFrom questions answered to insights gained,\nLangChain empowers, its purpose sustained.